**리더보드**<p>
* 평가 산식 : RMSE

<br>

**데이터 설명**

1. FIFA_train.csv / FIFA_test.csv<p>
 id : 선수 고유의 아이디<P>
 name : 이름<P>
 age : 나이<P>
 continent : 선수들의 국적이 포함되어 있는 대륙입니다<P>
 contract_until : 선수의 계약기간이 언제까지인지 나타내어 줍니다<P>
 position : 선수가 선호하는 포지션입니다. ex) 공격수, 수비수 등<P>
 prefer_foot : 선수가 선호하는 발입니다. ex) 오른발<P>
 reputation : 선수가 유명한 정도입니다. ex) 높은 수치일 수록 유명한 선수<P>
 stat_overall : 선수의 현재 능력치 입니다.<P>
 stat_potential : 선수가 경험 및 노력을 통해 발전할 수 있는 정도입니다.<P>
 stat_skill_moves : 선수의 개인기 능력치 입니다.<P>
 value : FIFA가 선정한 선수의 이적 시장 가격 (단위 : 유로) 입니다.<P>

<br>

2. submission.csv (제출 파일 형식)<p>
  id : 선수 고유의 아이디<p>
  value : 예측된 선수 이적 시장 가격을 기입 <p>

  <br>
  <br>

In [ ]:
# 드라이브 마운트

from google.colab import drive
drive.mount('/content/drive')

 # 1. 패키지 불러오기

# 2. 데이터 불러오기

In [ ]:
import pandas as pd

train = pd.read_csv('FIFA_train.csv')
test = pd.read_csv('FIFA_test.csv')
submission  = pd.read_csv('submission.csv')

# 3. 데이터 분석

# 4. 데이터 전처리

# 5. 모델 구현 및 적용

# 6. 성능개선